ML Component Models' Hyperparameter Tuning

In [7]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import random
from sklearn.preprocessing import RobustScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import ParameterGrid
import xgboost as xgb
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LSTM, Attention, Input, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
import matplotlib

# Set random seeds for reproducibility
random_seed = 42
np.random.seed(random_seed)
tf.random.set_seed(random_seed)
random.seed(random_seed)

# Ensure reproducibility with TensorFlow 2.x by setting deterministic operations
tf.keras.utils.set_random_seed(random_seed)
tf.config.experimental.enable_op_determinism()

# Create an output directory for graphs and other results
output_dir = "Segmented Approach Outputs"
os.makedirs(output_dir, exist_ok=True)

# Load and preprocess data
data = pd.read_csv("final_thesis_data.csv")
data['year_month'] = pd.to_datetime(data['year_month'])
data = data.sort_values(by=['country', 'year_month'])

# Aggregate data by country, year_month, sex, and age_group
data_agg = data.groupby(['country', 'year_month', 'sex', 'age_group']).sum().reset_index()

# Define variables to lag and create lagged variables
variables_to_lag = {
    "illegal_border_crossings": [3],
    "push_factor_index": [3],
    "push_factor_index_high_level": [2],
    "deaths_civilians": [12],
    "gdp_per_capita_current_usd": [12],
    "gdp_per_capita_growth": [12],
    "regime_end_type": [0],
    "state_fiscal_source_revenue": [0],
    "state_authority_over_territory": [0],
    "political_polarisation": [0],
    "political_violence": [0],
    "domestic_autonomy": [0],
    "rule_of_law": [0],
}

# Create lagged variables for the predictors
for var, lags in variables_to_lag.items():
    for lag in lags:
        lagged_var_name = f"{var}_lag_{lag}"
        data_agg[lagged_var_name] = data_agg.groupby(['country', 'sex', 'age_group'])[var].shift(lag)

# Define rolling window sizes
rolling_windows = [3, 6, 12, 24]

# Calculate rolling mean and standard deviation for asylum applications with different window sizes
for window in rolling_windows:
    data_agg[f'asy_applications_rolling_mean_{window}'] = data_agg.groupby(['country', 'sex', 'age_group'])['asy_applications'].transform(lambda x: x.rolling(window=window, min_periods=1).mean())
    data_agg[f'asy_applications_rolling_sd_{window}'] = data_agg.groupby(['country', 'sex', 'age_group'])['asy_applications'].transform(lambda x: x.rolling(window=window, min_periods=1).std())

# Calculate rolling mean and standard deviation for push_factor_index with different window sizes
for window in rolling_windows:
    data_agg[f'push_factor_index_rolling_mean_{window}'] = data_agg.groupby(['country', 'sex', 'age_group'])['push_factor_index'].transform(lambda x: x.rolling(window=window, min_periods=1).mean())
    data_agg[f'push_factor_index_rolling_sd_{window}'] = data_agg.groupby(['country', 'sex', 'age_group'])['push_factor_index'].transform(lambda x: x.rolling(window=window, min_periods=1).std())

# Calculate exponential moving average for asylum applications
data_agg['asy_applications_ewm_mean'] = data_agg.groupby(['country', 'sex', 'age_group'])['asy_applications'].transform(lambda x: x.ewm(span=12, adjust=False).mean())
data_agg['asy_applications_ewm_std'] = data_agg.groupby(['country', 'sex', 'age_group'])['asy_applications'].transform(lambda x: x.ewm(span=12, adjust=False).std())

# Add temporal features for seasonality
data_agg['month'] = data_agg['year_month'].dt.month
data_agg['month_sin'] = np.sin(2 * np.pi * data_agg['month'] / 12)
data_agg['month_cos'] = np.cos(2 * np.pi * data_agg['month'] / 12)

# Define the predictors and the target variable
predictors = [
    "illegal_border_crossings_lag_3", "push_factor_index_lag_3",
    "push_factor_index_high_level_lag_2", "deaths_civilians_lag_12",
    "gdp_per_capita_current_usd_lag_12", "gdp_per_capita_growth_lag_12",
    "regime_end_type_lag_0", "state_fiscal_source_revenue_lag_0",
    "state_authority_over_territory_lag_0", "political_polarisation_lag_0",
    "political_violence_lag_0", "domestic_autonomy_lag_0",
    "rule_of_law_lag_0",
    "asy_applications_rolling_mean_3", "asy_applications_rolling_sd_3",
    "asy_applications_rolling_mean_6", "asy_applications_rolling_sd_6",
    "asy_applications_rolling_mean_12", "asy_applications_rolling_sd_12",
    "asy_applications_rolling_mean_24", "asy_applications_rolling_sd_24",
    "push_factor_index_rolling_mean_3", "push_factor_index_rolling_sd_3",
    "push_factor_index_rolling_mean_6", "push_factor_index_rolling_sd_6",
    "push_factor_index_rolling_mean_12", "push_factor_index_rolling_sd_12",
    "push_factor_index_rolling_mean_24", "push_factor_index_rolling_sd_24",
    "asy_applications_ewm_mean", "asy_applications_ewm_std",
    "month_sin", "month_cos"
]
target_var = "asy_applications"

# Handle missing values using Iterative Imputer
imputer = IterativeImputer(max_iter=10, random_state=random_seed)
data_agg[predictors] = imputer.fit_transform(data_agg[predictors])

# Scale the predictors and target variable using RobustScaler
scaler_x = RobustScaler()
scaler_y = RobustScaler()
data_agg[predictors] = scaler_x.fit_transform(data_agg[predictors])
data_agg[target_var] = scaler_y.fit_transform(data_agg[target_var].values.reshape(-1, 1)).flatten()

# Split the data into training and validation sets
train_size = int(len(data_agg) * 0.8)
X_train, X_val = data_agg[predictors].iloc[:train_size], data_agg[predictors].iloc[train_size:]
y_train, y_val = data_agg[target_var].iloc[:train_size], data_agg[target_var].iloc[train_size:]

# Reshape data for LSTM (add time_steps dimension)
X_train_reshaped = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_val_reshaped = X_val.values.reshape((X_val.shape[0], 1, X_val.shape[1]))

KeyboardInterrupt: 

LSTM with Attention Model Hyperparameter Tuning

In [8]:
from sklearn.model_selection import ParameterGrid

# Define the hyperparameter grid for the LSTM model
lstm_param_grid = {
    'units_1': [16, 32, 64],
    'units_2': [16, 32, 64],
    'dropout_rate': [0.3, 0.5],
    'learning_rate': [0.001, 0.01],
    'l2_lambda': [0.001, 0.01]
}

def evaluate_lstm_model(params, X_train, y_train, X_val, y_val):
    model, _ = create_and_train_lstm_with_attention(
        X_train=np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1])),  # Reshape for LSTM
        y_train=y_train,
        X_val=np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1])),  # Reshape for LSTM
        y_val=y_val,
        units_1=params['units_1'],
        units_2=params['units_2'],
        dropout_rate=params['dropout_rate'],
        learning_rate=params['learning_rate'],
        l2_lambda=params['l2_lambda']
    )
    y_pred_val = model.predict(np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))).flatten()
    mse = mean_squared_error(y_val, y_pred_val)
    return mse

best_mse_lstm = float('inf')
best_params_lstm = None

for params in ParameterGrid(lstm_param_grid):
    mse = evaluate_lstm_model(params, X_train, y_train, X_val, y_val)
    if mse < best_mse_lstm:
        best_mse_lstm = mse
        best_params_lstm = params

print(f"Best LSTM Params: {best_params_lstm}, MSE: {best_mse_lstm}")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


 1/83 ━━━━━━━━━━━━━━━━━━━━ 14s 178ms/step

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (32, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


...

83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


 1/83 ━━━━━━━━━━━━━━━━━━━━ 11s 136ms/step

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (32, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Best LSTM Params: {'dropout_rate': 0.3, 'l2_lambda': 0.01, 'learning_rate': 0.001, 'units_1': 16, 'units_2': 32}, MSE: 6.2496734516308425


Feed-Forward Neural Network Model Hyperparameter Tuning

In [9]:
# Define the hyperparameter grid for the Neural Network model
nn_param_grid = {
    'units_1': [64, 128, 256],
    'units_2': [32, 64, 128],
    'dropout_rate': [0.2, 0.3, 0.4],
    'learning_rate': [0.001, 0.01],
    'l2_lambda': [0.001, 0.01]
}

def evaluate_nn_model(params, X_train, y_train, X_val, y_val):
    model, _ = create_best_nn_model(
        X_train=X_train,
        y_train=y_train,
        X_val=X_val,
        y_val=y_val,
        units_1=params['units_1'],
        units_2=params['units_2'],
        dropout_rate=params['dropout_rate'],
        learning_rate=params['learning_rate'],
        l2_lambda=params['l2_lambda']
    )
    y_pred_val = model.predict(X_val).flatten()
    mse = mean_squared_error(y_val, y_pred_val)
    return mse

best_mse_nn = float('inf')
best_params_nn = None

for params in ParameterGrid(nn_param_grid):
    mse = evaluate_nn_model(params, X_train, y_train, X_val, y_val)
    if mse < best_mse_nn:
        best_mse_nn = mse
        best_params_nn = params

print(f"Best NN Params: {best_params_nn}, MSE: {best_mse_nn}")


83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 711us/step
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 685us/step
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 835us/step
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 737us/step
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 862us/step
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 876us/step
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 995us/step
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 690us/step
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 691us/step
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 729us/step
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 814us/step
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 684us/step
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 687us/step
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 862us/step
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 717us/step
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 901us/step
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 669us/step
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 943us/step
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 691us/step
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 744us/step
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 951us/step
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 770us/step
83/83 ━━━━━━━━━━━━

XGBoost Model Hyperparameter Tuning

In [10]:
# Define the hyperparameter grid for the XGBoost model
xgb_param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'n_estimators': [50, 100, 200],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

def evaluate_xgb_model(params, X_train, y_train, X_val, y_val):
    model = xgb.XGBRegressor(
        learning_rate=params['learning_rate'],
        max_depth=params['max_depth'],
        n_estimators=params['n_estimators'],
        subsample=params.get('subsample', 1.0),
        colsample_bytree=params.get('colsample_bytree', 1.0),
        objective='reg:squarederror',
        random_state=random_seed,
        early_stopping_rounds=10
    )
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    y_pred_val = model.predict(X_val)
    mse = mean_squared_error(y_val, y_pred_val)
    return mse

best_mse_xgb = float('inf')
best_params_xgb = None

for params in ParameterGrid(xgb_param_grid):
    mse = evaluate_xgb_model(params, X_train, y_train, X_val, y_val)
    if mse < best_mse_xgb:
        best_mse_xgb = mse
        best_params_xgb = params

print(f"Best XGB Params: {best_params_xgb}, MSE: {best_mse_xgb}")


Best XGB Params: {'colsample_bytree': 1.0, 'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 50, 'subsample': 1.0}, MSE: 4.892468011184742
